# Getting Started with OpenAI Gym: Creatin Custom Environments
[From PaperspaceBlog](https://blog.paperspace.com/creating-custom-environments-openai-gym/)

In [53]:
!pip install opencv-python 
!pip install pillow

/bin/bash: /home/cim/anaconda3/envs/packing_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/cim/anaconda3/envs/packing_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [54]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random

from gym import Env, spaces
import time

font = cv2.FONT_HERSHEY_COMPLEX_SMALL 

It is a game environment where the agent is a Chopper pilot.
1. The chopper has to cover as much distance as possible to the mamzimum reward and avoid birds.
2. The episode terminates if there is a brid strike or the chopper runs out of fuel.
3. There are float fuel tanks which the chopper can collect to refuel to 1000 L (full capacity).

## Considerations
* The observation space can be discrete or continous.
* The action space can be discrete or continous.

*Prefer this to be continous because this will be the closest to our environment, but don't know how to implement.*

### ChopperScape Class

* Define in the `__init__` function the observation and the action spaces.
* Implement attribute `canvas` 
* Implement screen area `x_min, y_min, x_max, y_max`
* Implement `elements` - this stores elements in the screen like the chopper, bird, fuel station.
* Implement `max_fuel` to indicate the maximum fuel that the chopper can hold

In [55]:
class ChopperScape(Env):
    def __init__(self, name, x_l = 600, y_l= 600, icon_w = 40, icon_h= 40):
        super(ChopperScape, self).__init__()

        # Define 2D observation space
        self.observation_shape = (x_l, y_l, 3) # not sure what the 3 does
        # Creates a continuous space (dtype = float64), with the low at zero and 
        # the high at 1 in the shape of the observation space
        self.observation_space = spaces.Box(low = np.zeros(self.observation_shape), 
                                            high = np.ones(self.observation_shape),
                                            dtype = np.float64)

        # Define an action space ranging for 0 to 4
        self.action_space = spaces.Discrete(6,) # Would like this to be continous

        # Create a canvas to render the environment images upon
        self.canvas = np.ones(self.observation_shape) * 1 # Why multiply by 1?

        # Define elements present inside the enviroment
        self.elements = []

        # Maximum fuel the chopper can take
        self.max_fuel = 1000

        # Permissible area where the helicopter can be located (helicopter stays on left side of panel)
        self.y_min = int(self.observation_shape[0] * 1)
        self.x_min = 0
        self.y_max = int(self.observation_shape[0] * 1)
        self.x_max = self.observation_shape[1]
        self.icon_w = icon_w
        self.icon_h = icon_h

    # Define helper fuunction to draw on Canvas
    def draw_elements_on_canvas(self):
        # Init toe canvas
        self.canvas = np.ones(self.observation_shape) * 1
        # Draw the helicopter on the canvas
        for elem in self.elememts:
            elem_shape = elem.icon.shape
            x,y = elem.x, elem.y
            self.canvas[y : y + elem_shape[1], x : x + elem_shape[0]] = elem.icon

    # Defining the reset condition
    def reset(self):

    # Number of birds
        self.bird_count = 0

        # Initialise the chopper
        self.chopper = Chopper('chopper', self.x_max, self. x_min, 
                                self.y_max, self.y_min, self.icon_w, self.icon_h)    
        self.chopper.set_position(x, y)

        # Reset the canvas
        self.canvas = np.ones(self.observation_shape) * 1

        # draw elements on canvas
        self.draw_elements_on_canvas()

        # return the observation
        return self.canvas



        

### Elements of the Environment
3 distinct elements
* Chopper
* Birds
* Fuel Stations (floating)

Implement all of this as separate classes that inherit from a common base class called `Point`.

### Point Base Class
`Point` class is used to define any arbitrary point in the observation image. Define it with the following attributes:
* `(x, y)`: position of the point on the image.
* `(x_min, x_max, y_min, y_max)`: the permissible coordicates for the points - keeps the position values clamped to these limits. (We can use this to define our box).
* `name`: name of the point.

The following member functions for this class are defined as:
* `get_position`: get the coordinate of the point.
* `set_position`: set the coordinate of the point to a certain value.
* `move`: move the points by a certain value.

In [56]:
class Point(object):
    def __init__(self, name, x_max, x_min, y_max, y_min, icon_w, icon_h):
        self.x = 0
        self.y = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.icon_w = icon_w
        self.icon_h = icon_h
    
    def set_position(self, x, y):
        # icon_w is width of object character on screen
        self.x = self.clamp(x, self.x_min, self.x_max - self.icon_w) 
        # icon_h is height of object character on screen
        self.y = self.clamp(y, self.y_min, self.y_max - self.icon_h) 
        # We would probably have to set the icon_w and icon_h to box dimensions

    def get_position(self):
        return (self.x, self.y)

    def move(self, del_x, del_y):
        self.x += del_x
        self.y += del_y

        self.x = self.clamp(self.x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(self.y, self.y_min, self.y_max - self.icon_h)

    def clamp(self, n, minn, maxn):
        return (max(min(maxn, n), minn))

Now there is a general `Point` class to construct all of the objects in the environment.

### More items to add to ChopperScape Class
Objects
* `Chopper`
* `Bird`
* `Fuel`

Also will need to add
* `reset`
* `step`

The objects will inherit from the `Point` class and include some new attributes:
* `icon` - the 2 dimensional icon that will display on the observation image.
* `(icon_w, icon_h)` - the dimensions of the icon.  -- *We can use the dimensions of the box we're placing.*

In [57]:
class Chopper(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min, icon_w, icon_h):
        super(Chopper, self).__init__( name, x_max, x_min, y_max, y_min, icon_w, icon_h)
        self.icon = cv2.imread('chopper.png') / 255.0 # I think the 255.0 is for gray scale
        icon_w = 64 # we will set this as a variable we can pass into the function
        icon_h = 64 # again we will set this as a variable
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

class Bird(Point):
    def __inti__(self, name, x_max, x_min, y_max, y_min):
        super(Bird, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread('bird.png') / 255.0
        self.icon_w = 32
        self.icon_h = 32
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

class Fuel(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Fuel, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread('fuel.png') / 255.0
        self.icon_w = 32
        self.icon_h = 32
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

#### Adding to the `Chopper` class
1. `reset` : resets the environment to its initial state and returns the initial observation
2. `step` : Executes a step by applying an action in the environment.   
    * Returns the new observation
    * reward
    * completion status
    * and other information

#### Reset Function
Returns the chopper to the initial state - it is initialed randomly in an area in the top-left of our screen.

In [58]:
# env = ChopperScape('test')
# obs = env.reset()
# plt.imshow(obs)

[ WARN:0@54782.273] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('chopper.png'): can't open/read file: check file path/integrity


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [59]:
env = gym.make('Bpp-v0')

UnregisteredEnv: No registered env with id: Bpp-v0